In [ ]:
from kamiwaza.services.models.services import ModelService, ModelFileService
from kamiwaza.services.models.schemas import Model, ModelFile, ModelDownloadRequest
import logging
import platform
import re
import ray

# Set up logging
logging.basicConfig(level=logging.WARNING)
logging.getLogger("requests").setLevel(logging.WARNING)
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.WARNING)

## Getting a Model

Important note: many of Kamiwaza's services are exposed as REST endpoints, allowing you to build applications that leverage Kamiwaza features from other systems.

Most of the functions of ModelService are available in the /models namespace. See the REST docs. (Link in the GUI to full swagger spec)

In [ ]:
model_service = ModelService()
model_file_service = ModelFileService()

In [ ]:
# Define target model patterns based on the operating system
if platform.system() == 'Darwin':
    TARGET_MODEL_PATTERN = ('llama3.1-8b*instruct', 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF')
elif platform.system() == 'Linux':
    TARGET_MODEL_PATTERN = ('llama3.1-8b*instruct', 'hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4')
else:
    raise ValueError(f"Unsupported operating system: {platform.system()}")

In [ ]:
def find_and_select_model(pattern, repo_id=None):
    """
    Find and select a model based on the given pattern and repository ID.
    
    Args:
        pattern (str): The pattern to match for model names.
        repo_id (str, optional): The specific repository ID to check.
    
    Returns:
        Model: The selected model object.
    
    Raises:
        ValueError: If no suitable model is found.
    """
    # Step 1: Check for locally existing models
    local_models = model_service.list_models(load_files=True)
    for model in local_models:
        if re.search(pattern, model.name) or (repo_id and model.repo_modelId == repo_id):
            print(f"Found existing local model: {model.name}")
            return model

    # Step 2: Try exact match from hub if repo_id is provided
    if repo_id:
        hub_models = model_service.search_models(query=repo_id, hubs_to_search=['*'], exact=True)
        if hub_models:
            print(f"Found exact match on hub: {hub_models[0].name}")
            return hub_models[0]

    # Step 3: Fuzzy search on hub
    search_pattern = f"*{pattern}*" if pattern[0] != '^' and pattern[-1] != '$' else pattern
    hub_models = model_service.search_models(query=search_pattern, hubs_to_search=['*'])
    
    # Filter models based on the operating system
    filtered_models = []
    for model in hub_models:
        if platform.system() == 'Darwin' and 'GGUF' in model.name:
            filtered_models.append(model)
        elif platform.system() == 'Linux' and 'GGUF' not in model.name and ('AWQ' in model.name or 'GPTQ' in model.name):
            filtered_models.append(model)
    
    if filtered_models:
        # Select the first model from the filtered list
        selected_model = filtered_models[0]
        print(f"Selected model from hub: {selected_model.name}")
        return selected_model
    
    raise ValueError(f"No suitable model found for pattern: {pattern}")

# Use the function to select the model
selected_model = find_and_select_model(TARGET_MODEL_PATTERN[0], TARGET_MODEL_PATTERN[1])


In [ ]:
selected_model = find_and_select_model(TARGET_MODEL_PATTERN[0], TARGET_MODEL_PATTERN[1])

In [ ]:
if not isinstance(selected_model, Model) or not selected_model.id:
    # Model is not local or doesn't have an ID, so we need to download it
    files = model_service.list_hub_model_files(hub=selected_model.hub, model_id=selected_model.repo_modelId)
    
    if platform.system() == 'Darwin':
        # For macOS, we download one GGUF file (prioritizing q6_k, q5_k_m, q4_k_m) and all non-GGUF files
        gguf_files = [file for file in files if file.name.lower().endswith('.gguf')]
        priority_order = ['q6_k', 'q5_k_m', 'q4_k_m']
        
        # Select the first GGUF file that matches the priority order, or the first GGUF file if none match
        selected_gguf = next((file for priority in priority_order 
                              for file in gguf_files if priority in file.name.lower()), 
                             gguf_files[0] if gguf_files else None)
        
        if selected_gguf:
            files_to_download = [file.name for file in files if not file.name.lower().endswith('.gguf')] + [selected_gguf.name]
        else:
            raise ValueError("No GGUF files found for macOS. Cannot proceed with download.")
    
    elif platform.system() == 'Linux':
        # For Linux, we download all files except GGUF files
        files_to_download = [file.name for file in files if not file.name.lower().endswith('.gguf')]
    
    else:
        raise ValueError(f"Unsupported operating system: {platform.system()}")
else:
    # Model exists locally, so we'll re-download files marked for download or with non-empty storage location
    files_to_download = [
        file.name for file in selected_model.m_files 
        if file.download or (file.storage_location and file.storage_location.strip())
    ]

print(f"Files selected for download: {', '.join(files_to_download)}")

# Proceed with download
if files_to_download:
    result = model_service.download_model_files(
        model_id=selected_model.repo_modelId,
        hub=selected_model.hub,
        files_to_download=files_to_download
    )
    print(f"Download result: {result}")
else:
    print("No files need to be downloaded.")
    result = {'files': []}

# Ensure we have a valid model_id for future use
model_id = selected_model.id if isinstance(selected_model, Model) else selected_model.repo_modelId
print(f"Model ID: {model_id}")

In [ ]:
if not isinstance(selected_model, Model):
    download_status = model_file_service.get_download_status([selected_model.id])
    for status in download_status:
        print(f"File: {status.name}, Progress: {status.download_percentage}%, Status: {'Downloading' if status.is_downloading else 'Pending'}")


In [ ]:
if result and 'files' in result and result['files']:
    print("Downloaded Model Files:")
    for file_id in result['files']:
        model_file = model_file_service.get_model_file(model_file_id=str(file_id))
        print(f"\nFile: {model_file.name}")
        print(f"  ID: {model_file.id}")
        print(f"  Download Requested At: {model_file.dl_requested_at}")
        print(f"  Size: {model_file.size} bytes")
        print(f"  Storage Location: {model_file.storage_location}")
        print(f"  Checksum: {model_file.checksum}")
        print(f"  Checksum Type: {model_file.checksum_type}")
    
    # Set model_id based on the first file (assuming all files belong to the same model)
    model_id = model_file_service.get_model_file(model_file_id=str(result['files'][0])).m_id
    print(f"\nModel ID: {model_id}")
else:
    print("No files were downloaded or the download result is invalid.")
    model_id = None  # Set model_id to None if no files were downloaded

# Validate model_id
if model_id is None:
    raise ValueError("Failed to obtain a valid model ID. Please check the download process and try again.")

# Background or Notebook download?

If you have the Kamiwaza service running, the downloads will be going now. We assume this is the default.

You can trigger the download manually, calling `model_service.process_downloads()`; by default, Kamiwaza does up to 6 at a time. This actually creates subprocesses, because we read the download prorgress to update internal state.

We'll assume the service is running, but process_downloads() also exits after creating download workers, so you can, if needed, add it and then continue with this code

**Note**: Kamiwaza doesn't re-download model files in general (if the checksum matches, or the size if a checksum isn't available, which is the case for small files on HF); so you can re-download this; the call will still execute, but it will evaluate checksums and then exit

In [ ]:
import time
from IPython.display import clear_output

while True:
    downloads = model_file_service.get_download_status(model_ids=[model_id])
    if len(downloads) < 1:
        break
    for d in downloads:
        print(f"{d.name}:")
        print(f"  Progress: {d.download_percentage}%")
        print(f"  Elapsed: {d.download_elapsed}")
        print(f"  Remaining: {d.download_remaining}")
        print(f"  Throughput: {d.download_throughput}")
        print(f"  PID: {d.download_pid}")
        print("--------------------")
    time.sleep(2)
    clear_output(wait=True)
print("Download complete")

## IMPORTANT NOTE

If you are on OSX, you must have built llama.cpp with with the `bash build-llama-cpp.sh` command as per `RELEASE.md` and `kamiwaza/README.md`; if you have not, the serving service will not find llamacpp to use and this will cause an error.

(If you are on Linux your engine will be vLLM and deploy as a container. OSX does not provide Docker GPU access, so we can't go doing that.)

In [ ]:
from kamiwaza.serving.schemas.serving import CreateModelDeployment
from kamiwaza.serving.services import ServingService

# Get the default model config
configs = model_service.get_model_configs(model_id=model_id)
default_config = next((config for config in configs if config.default), configs[0])

if not ray.is_initialized():
    # in <= 0.3.1 the kamiwaza deployer requires ray to be initialized for ray.serve to wrapper the deployment, and this is "prod-like" so we will handle it.
    # (also, ray in jupyter is very noisy)
    ray.init(address="auto", configure_logging=True, log_to_driver=False, logging_level=logging.ERROR, logging_config=ray.LoggingConfig(encoding="TEXT", log_level="ERROR"))
    
# Create deployment request
deployment_request = CreateModelDeployment(
    m_id=model_id,
    m_config_id=default_config.id,
    m_file_id=model_file.id,
    autoscaling=True,
    min_copies=1,
    max_copies=3,
    max_concurrent_requests=32
)

# Deploy the model
serving_service = ServingService()
deployment_id = await serving_service.deploy_model(deployment_request)
print(f"Deployment ID: {deployment_id}")

## Notes

In 0.3.0+ URLs can be constructed in various ways; lb_port is key as each host will listen on the same port; host_name can be acquired from the model_instance and yet is less matterful. In production, we recommend setting a DNS entry for model access that round-robins the cluster; the Kamiwaza load balancer will route to model instances (which is where host_name does matter, as we want to land requests on the host or hosts where the replicas are deployed)

Note that the default interface on llama.cpp is the llama cpp web interface, but the OpenAI chatcompletion api endpoint is accessible on both

engines if you specify the hostname and port to the openai client. (Tested only with openai>=0.28.0,<1.0, the version before "conversations" were introduced)

In [ ]:
import time

time.sleep(5)  # Wait a bit for the deployment to initialize

deployments = serving_service.list_deployments(with_names=True)
valid_deployments = []
for deployment in deployments:
    if deployment.status == 'STOPPED':
        continue
    if not deployment.instances:
        continue

    valid_deployments.append(deployment)
    print(f"Deployment ID: {deployment.id}")
    print(f"Model Name: {deployment.m_name}")
    print(f"Status: {deployment.status}")
    print(f"Engine: {deployment.engine_name}")
    
    # Use the lb_port directly from the deployment
    print(f"Load Balancer Port: {deployment.lb_port}")

    # Any host in the cluster in general should work
    target_host = "localhost"

    # This host name detection is not a prod pattern
    if deployment.instances and deployment.instances[0].host_name:
        target_host = deployment.instances[0].host_name
    
    print(f"API URL: http://{target_host}:{deployment.lb_port}")
    print(f"\tNote: the API URL will be a 404 at the base; but an example working endpoint will typically be http://{target_host}:{deployment.lb_port}/v1/models")
    
    if deployment.engine_name == 'vllm':
        print(f"API Docs: http://{target_host}:{deployment.lb_port}/docs")
    elif deployment.engine_name == 'llamacpp':
        print(f"Llamacpp Chat: http://{target_host}:{deployment.lb_port}")
    
    print("Instances:")
    for instance in deployment.instances:
        print(f"  - Host: {instance.host_name}, Port: {instance.listen_port}, Status: {instance.status}")
    
    print("--------------------")

if not valid_deployments:
    print("No deployments found.")


## Note!

If you get a 404/other error, you should check the host URL. Given the mix of notebooks, prod, NAT and not, cloud, VPNs, the above uri may not always be correct.

## Done! Move on and come back to run this later (or use the UI)

You can move on to the inference notebook. You can run the cell below after removing the comment to programmatically stop the endpoint; or use the UI

Models->Model Details (click the model name) -> Details (on the model deployment bottom of right side column) -> Stop Deployment


In [ ]:
import asyncio
import socket

# Get the hostname to use in URLs
target_host = socket.gethostname()

async def monitor_and_stop_deployment(deployment_id):
    while True:
        deployment = serving_service.get_deployment(deployment_id)
        print(f"Deployment Status: {deployment.status}")
        if deployment.status == 'DEPLOYED':
            print("Deployment is ready.")
            break
        elif deployment.status in ['FAILED', 'ERROR']:
            print("Deployment failed.")
            break
        await asyncio.sleep(5)
    
    # Uncomment the following lines to automatically stop the deployment
    # print("Stopping deployment...")
    # await serving_service.stop_deployment(deployment_id)
    # print("Deployment stopped.")

# Run the monitoring function
await monitor_and_stop_deployment(deployment_id)

# List final deployment status
final_deployments = serving_service.list_deployments(with_names=True)
if not final_deployments:
    print("No deployments running!")
else:
    for deployment in final_deployments:
        if deployment.status == 'STOPPED':
            continue
        if not deployment.instances:
            continue
        print(f"Deployment ID: {deployment.id}")
        print(f"Model Name: {deployment.m_name}")
        print(f"Status: {deployment.status}")
        print(f"Load Balancer Port: {deployment.lb_port}")
        print(f"API URL: http://{target_host}:{deployment.lb_port}")
        
        if deployment.engine_name == 'vllm':
            print(f"API Docs: http://{target_host}:{deployment.lb_port}/docs")
        elif deployment.engine_name == 'llamacpp':
            print(f"Llamacpp Chat: http://{target_host}:{deployment.lb_port}")
        
        print("Instances:")
        if deployment.instances:
            for instance in deployment.instances:
                print(f"  - Host: {instance.host_name}, Port: {instance.listen_port}, Status: {instance.status}")
        else:
            print("  No instances available")
        
        print("--------------------")

## Final Note

As a side note, the stop deployment operation is blocking; but we do set the status to **STOP_REQUESTED** before we attempt to terminate the deployment